In [27]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.stats import norm, beta
from IPython.display import display
import math

# Function to calculate lower, upper, and mode for PERT distribution
def calculate_pert_parameters(mean, std_dev):
    # Define lower limit for symmetric distribution ensuring non-negative lower limit
    d = math.sqrt(7.0) * std_dev
    lower_limit = max(0, mean - d)
    
    # Define upper limit to ensure the relative standard deviation is met
    upper_limit = mean + 7 * std_dev**2 / (mean - lower_limit)

    # Recalculate mode
    mode = (6 * mean - lower_limit - upper_limit) / 4.0
    return lower_limit, upper_limit, mode

# Function to adjust sharpness to align mode density of PERT with Normal distribution's mean density
def calculate_sharpness(mean, std_dev, lower_limit, upper_limit):
    normal_density_at_mean = norm.pdf(mean, mean, std_dev)
    # Adjust sharpness until the PERT density at the mode equals the normal density at the mean
    sharpness = 1.0
    tolerance = 1e-5
    max_iterations = 100
    for _ in range(max_iterations):
        a = 1 + 4 * (mean - lower_limit) / (upper_limit - lower_limit) * sharpness
        b = 1 + 4 * (upper_limit - mean) / (upper_limit - lower_limit) * sharpness
        pert_density_at_mean = beta.pdf(mean, a, b, loc=lower_limit, scale=upper_limit - lower_limit)
        if abs(pert_density_at_mean - normal_density_at_mean) < tolerance:
            break
        sharpness *= 1.1 if pert_density_at_mean < normal_density_at_mean else 0.9
    return sharpness

# Function to generate the PDFs
def plot_pdfs(rel_std_dev, uniformity, sharpness):
    mean = 1.0  # Fixed mean value
    std_dev = rel_std_dev * mean

    # Calculate PERT parameters
    lower_limit_pert, upper_limit_pert, mode_pert = calculate_pert_parameters(mean, std_dev)

    # Calculate sharpness to align mode density of PERT with Normal distribution's mean density if sharpness slider is at 0
    if sharpness == 0:
        sharpness = calculate_sharpness(mean, std_dev, lower_limit_pert, upper_limit_pert)

    # Define limits for the x-axis (always from 0 to 2)
    x_pert = np.linspace(0, 2, 1000)
    x_normal = np.linspace(0, 2, 1000)

    # Normal Distribution
    normal_pdf = norm.pdf(x_normal, mean, std_dev)

    # Transition from PERT to Uniform
    # When uniformity is 0, it is a standard PERT distribution.
    # When uniformity is 1, it becomes a uniform distribution.
    # The sharpness parameter is calculated to adjust the alpha and beta values to make the distribution sharper.
    a = 1 + (4 * (mean - lower_limit_pert) / (upper_limit_pert - lower_limit_pert)) * (1 - uniformity) * sharpness
    b = 1 + (4 * (upper_limit_pert - mean) / (upper_limit_pert - lower_limit_pert)) * (1 - uniformity) * sharpness
    pert_pdf = beta.pdf(x_pert, a, b, loc=lower_limit_pert, scale=upper_limit_pert - lower_limit_pert)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(x_normal, normal_pdf, label='Normal Distribution', color='blue')
    plt.plot(x_pert, pert_pdf, label='PERT to Uniform Distribution', color='red')
    plt.title(f'PDFs with Relative Standard Deviation: {rel_std_dev:.2f}, Mean: {mean:.2f}, Uniformity: {uniformity:.2f}, Sharpness: {sharpness:.2f}')
    plt.xlabel('X')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 2)  # Set x-axis limit from 0 to 2
    plt.show()

# Slider widgets for relative standard deviation, uniformity, and sharpness
rel_std_dev_slider = widgets.FloatSlider(value=0.5, min=0.05, max=2.0, step=0.05, description='Rel Std Dev:')
uniformity_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, description='Uniformity:')
sharpness_slider = widgets.FloatSlider(value=0.0, min=0.0, max=5.0, step=0.1, description='Sharpness:')

# Interactive widget to update the plot
ui = widgets.VBox([rel_std_dev_slider, uniformity_slider, sharpness_slider])
output = widgets.interactive_output(plot_pdfs, {'rel_std_dev': rel_std_dev_slider, 'uniformity': uniformity_slider, 'sharpness': sharpness_slider})

# Display the sliders and the plot
display(ui, output)


Output()